In [ ]:
!pip install pyngrok==6.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=582ddb61c2894864854dd9df332f47abbafce4c751146ef632cbe215a6ae2feb
  Stored in directory: /root/.cache/pip/wheels/b6/2f/3d/4957f583fdfd1c359843d2e57981c634692c579788b2dfc088
Successfully built pyngrok


In [ ]:
!pip install flask torch opencv-python numpy librosa ffmpeg-python pyngrok

In [ ]:
# Install required libraries
!pip install flask pyngrok gdown pydub

# Clone the repository
!git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git

# Set the base path
basePath = "/content/Wav2Lip-GFPGAN"
%cd {basePath}

# Define folder names
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'

# Define paths
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

# Download the pretrained model for face detection
!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}/face_detection/detection/sfd/s3fd.pth

# Download the Wav2Lip pretrained model
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}/checkpoints/wav2lip.pth

# Install dependencies
!pip install -r requirements.txt

# Import necessary libraries
from flask import Flask, request, jsonify, send_file
import os
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from concurrent.futures import ProcessPoolExecutor
import subprocess
from pydub import AudioSegment

# Initialize Flask app
app = Flask(__name__)

# Set up paths
inputPath = basePath + '/inputs'
outputPath = basePath + '/outputs'

# Create input and output directories if they don’t exist
os.makedirs(inputPath, exist_ok=True)
os.makedirs(outputPath, exist_ok=True)

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'mp4', 'wav'}

# Function to check if a file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Initialize process pool with max_workers=3 to limit concurrent inference processes
executor = ProcessPoolExecutor(max_workers=3)

# Function to detect leading silence
def detect_leading_silence(sound, silence_threshold=-40.0, chunk_size=10):
    trim_ms = 0
    assert chunk_size > 0
    while sound[trim_ms:trim_ms + chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size
    return trim_ms

# Function to trim leading and trailing silence from audio
def trim_leading_trailing_silence(audio_path):
    audio = AudioSegment.from_wav(audio_path)
    start_trim = detect_leading_silence(audio)
    end_trim = detect_leading_silence(audio.reverse())
    duration = len(audio)
    trimmed_audio = audio[start_trim:duration - end_trim]
    trimmed_audio_path = audio_path.replace('.wav', '_trimmed.wav')
    trimmed_audio.export(trimmed_audio_path, format='wav')
    return trimmed_audio_path

# Function to run Wav2Lip inference without resizing
def run_inference(video_path, audio_path, output_path):
    cmd = [
        'python', 'inference.py',
        '--checkpoint_path', 'checkpoints/wav2lip.pth',
        '--face', video_path,
        '--audio', audio_path,
        '--outfile', output_path
    ]
    subprocess.run(cmd, cwd=wav2lipPath, check=True)

# Flask route to handle file upload and processing
@app.route('/process', methods=['POST'])
def process_files():
    # Check if files are present in the request
    if 'video' not in request.files or 'audio' not in request.files:
        return jsonify({"error": "Both video and audio files are required"}), 400

    video_file = request.files['video']
    audio_file = request.files['audio']

    # Check if files have valid extensions
    if not allowed_file(video_file.filename) or not allowed_file(audio_file.filename):
        return jsonify({"error": "Invalid file type. Only .mp4 and .wav files are allowed"}), 400

    # Save the uploaded files with unique filenames to avoid overwriting
    video_filename = secure_filename(video_file.filename)
    audio_filename = secure_filename(audio_file.filename)
    video_path = os.path.join(inputPath, video_filename)
    audio_path = os.path.join(inputPath, audio_filename)
    video_file.save(video_path)
    audio_file.save(audio_path)

    # Trim leading and trailing silence from audio
    audio_path = trim_leading_trailing_silence(audio_path)

    # Define a unique output path for the lip-synced video
    output_filename = f"result_{os.path.splitext(video_filename)[0]}_{os.path.splitext(audio_filename)[0]}.mp4"
    lipSyncedOutputPath = os.path.join(outputPath, output_filename)

    # Submit the inference task to the process pool and wait for completion
    future = executor.submit(run_inference, video_path, audio_path, lipSyncedOutputPath)
    future.result()  # Blocks until the task is complete

    # Return the output file as a downloadable link
    return send_file(lipSyncedOutputPath, as_attachment=True)

# Start the Flask server
if __name__ == '__main__':
    # Set your ngrok token here
    ngrok.set_auth_token("2onJMt9ol5PDI1N4NeEUjrNyh3y_2DjZnzRrQwKo39tJrRt9k")

    # Expose the Flask app via ngrok
    public_url = ngrok.connect(5000).public_url
    print(f" * Running on {public_url}")

    # Run the Flask app with threading enabled
    app.run(host='0.0.0.0', port=5000, threaded=True)